In [25]:
import pandas as pd

In [26]:
dataPayment = '../Datasets/Employee_Payroll.csv'
payroll = pd.read_csv(dataPayment)
payroll.head()

,Fiscal Year,Fiscal Quarter,Fiscal Period,First Name,Last Name,Middle Init,Bureau,Office,Office Name,Job Code,Job Title,Base Pay,Position ID,Employee Identifier,Original Hire Date
0,2016,1,2016Q1,AMRITH,AAKRE,K,State's Attorney,1250.0,STATES ATTORNEY,1172,Assistant State's Attorney,20088.00,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,05/16/2005
1,2016,2,2016Q2,AMRITH,AAKRE,K,State's Attorney,1250.0,STATES ATTORNEY,1172,Assistant State's Attorney,23436.00,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,05/16/2005
2,2016,3,2016Q3,AMRITH,AAKRE,K,State's Attorney,1250.0,STATES ATTORNEY,1172,Assistant State's Attorney,20422.82,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,05/16/2005
3,2016,4,2016Q4,AMRITH,AAKRE,K,State's Attorney,1250.0,STATES ATTORNEY,1172,Assistant State's Attorney,23904.80,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,05/16/2005
4,2017,1,2017Q1,AMRITH,AAKRE,K,State's Attorney,1250.0,STATES ATTORNEY,1172,Assistant State's Attorney,20745.80,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,05/16/2005


In [27]:
payroll.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234299 entries, 0 to 234298
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Fiscal Year          234299 non-null  int64  
 1   Fiscal Quarter       234299 non-null  int64  
 2   Fiscal Period        234299 non-null  object 
 3   First Name           234299 non-null  object 
 4   Last Name            234299 non-null  object 
 5   Middle Init          140887 non-null  object 
 6   Bureau               234299 non-null  object 
 7   Office               231115 non-null  float64
 8   Office Name          231115 non-null  object 
 9   Job Code             234299 non-null  int64  
 10  Job Title            234299 non-null  object 
 11  Base Pay             234295 non-null  float64
 12  Position ID          234299 non-null  int64  
 13  Employee Identifier  234299 non-null  object 
 14  Original Hire Date   234299 non-null  object 
dtypes: float64(2), in

In [28]:
payroll.pop('First Name')

0          AMRITH
1          AMRITH
2          AMRITH
3          AMRITH
4          AMRITH
           ...   
234294      NORMA
234295      FRANK
234296    CYNTHIA
234297      LAURA
234298      NAKIA
Name: First Name, Length: 234299, dtype: object

In [29]:
payroll.head()

,Fiscal Year,Fiscal Quarter,Fiscal Period,Last Name,Middle Init,Bureau,Office,Office Name,Job Code,Job Title,Base Pay,Position ID,Employee Identifier,Original Hire Date
0,2016,1,2016Q1,AAKRE,K,State's Attorney,1250.0,STATES ATTORNEY,1172,Assistant State's Attorney,20088.00,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,05/16/2005
1,2016,2,2016Q2,AAKRE,K,State's Attorney,1250.0,STATES ATTORNEY,1172,Assistant State's Attorney,23436.00,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,05/16/2005
2,2016,3,2016Q3,AAKRE,K,State's Attorney,1250.0,STATES ATTORNEY,1172,Assistant State's Attorney,20422.82,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,05/16/2005
3,2016,4,2016Q4,AAKRE,K,State's Attorney,1250.0,STATES ATTORNEY,1172,Assistant State's Attorney,23904.80,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,05/16/2005
4,2017,1,2017Q1,AAKRE,K,State's Attorney,1250.0,STATES ATTORNEY,1172,Assistant State's Attorney,20745.80,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,05/16/2005


In [33]:
pr = payroll.loc[payroll['Job Code'] == 1172]
pr

,Fiscal Year,Fiscal Quarter,Fiscal Period,Last Name,Middle Init,Bureau,Office,Office Name,Job Code,Job Title,Base Pay,Position ID,Employee Identifier,Original Hire Date
0,2016,1,2016Q1,AAKRE,K,State's Attorney,1250.0,STATES ATTORNEY,1172,Assistant State's Attorney,20088.00,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,05/16/2005
1,2016,2,2016Q2,AAKRE,K,State's Attorney,1250.0,STATES ATTORNEY,1172,Assistant State's Attorney,23436.00,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,05/16/2005
2,2016,3,2016Q3,AAKRE,K,State's Attorney,1250.0,STATES ATTORNEY,1172,Assistant State's Attorney,20422.82,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,05/16/2005
3,2016,4,2016Q4,AAKRE,K,State's Attorney,1250.0,STATES ATTORNEY,1172,Assistant State's Attorney,23904.80,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,05/16/2005
4,2017,1,2017Q1,AAKRE,K,State's Attorney,1250.0,STATES ATTORNEY,1172,Assistant State's Attorney,20745.80,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,05/16/2005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234141,2018,1,2018Q1,SIMPSON,P,STATES ATTORNEY,1250.0,STATES ATTORNEY,1172,Assistant State's Attorney,26268.82,9510272,fb4194e5-0b14-4019-b0eb-7eae796c8cc4,08/21/1997
234146,2018,2,2018Q2,SCADUTO,J,STATES ATTORNEY,1250.0,STATES ATTORNEY,1172,Assistant State's Attorney,28039.76,9509780,4fca97bc-ed5d-4ab9-b130-d0587d222594,03/29/1993
234177,2018,1,2018Q1,COTTER,S,STATES ATTORNEY,1250.0,STATES ATTORNEY,1172,Assistant State's Attorney,23247.67,9509854,21703b65-7357-45b0-a256-cc06393750d5,08/09/2004
234179,2018,2,2018Q2,MANASTER,NaN,STATES ATTORNEY,1250.0,STATES ATTORNEY,1172,Assistant State's Attorney,25252.64,9509965,c3b70364-77a0-4300-be9f-11bf5e869c87,02/14/2000


In [35]:
len(pr.axes[0])

7603

In [44]:
_index = payroll['Job Code'].unique()

_ids = []

for _id in _index:
    counts = len(payroll.groupby(['Job Code']).get_group(_id))
    if counts > 1000:
        _ids.append(_id)
        
print(len(_ids))

38


In [46]:
# pr = payroll.loc[payroll['Job Code'] == 1172]
for x in _ids:
    df = payroll.loc[payroll['Job Code'] == x]
    df.to_csv('../Datasets/' + str(x) + '.csv')